In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [7]:
img_height = 128
img_width = 128
batch_size = 5

In [8]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('/kaggle/input/isl-sls/data1/train',
                                                 class_mode = 'categorical',
                                                 batch_size = batch_size,
                                                 target_size = (img_height, img_width),
                                                 shuffle = True,
                                                 color_mode='grayscale',
                                                 )

test_set = test_datagen.flow_from_directory('/kaggle/input/isl-sls/data1/test',
                                            class_mode = 'categorical',
                                            batch_size = batch_size,
                                            target_size = (img_height, img_width),
                                            shuffle = True,
                                            color_mode='grayscale',
                                            )

Found 12150 images belonging to 27 classes.
Found 4050 images belonging to 27 classes.


In [9]:
model = Sequential() #a sequential cnn model to create

#added a neuron to network
model.add(Conv2D(32,(2,2), input_shape=(128, 128, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(64,(2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(128 ,(2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

#flattening image data
model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(27))#extra 1 output to find if no alphabet is detected 26 alphabets
model.add(Activation('softmax'))#activation function

In [10]:
train_set_size = training_set.samples
test_set_size = test_set.samples
print("Number of images in the training set:", train_set_size)
print("Number of images in the test set:", test_set_size)

Number of images in the training set: 12150
Number of images in the test set: 4050


In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-7)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 127, 32)      160       
                                                                 
 activation (Activation)     (None, 127, 127, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 42, 42, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 41, 41, 64)        8256      
                                                                 
 activation_1 (Activation)   (None, 41, 41, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                    

In [12]:
history = model.fit(training_set, epochs=2, validation_data=test_set, verbose=1)

Epoch 1/2
2430/2430 [==============================] - 99s 36ms/step - loss: 0.4293 - accuracy: 0.8714 - val_loss: 0.0022 - val_accuracy: 0.9993
Epoch 2/2
2430/2430 [==============================] - 54s 22ms/step - loss: 0.0320 - accuracy: 0.9900 - val_loss: 3.2684e-05 - val_accuracy: 1.0000


In [13]:
score = model.evaluate(test_set, steps=len(test_set))
print('Test accuracy:', score[1])

810/810 [==============================] - 8s 10ms/step - loss: 3.2684e-05 - accuracy: 1.0000
Test accuracy: 1.0


In [15]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
model.save_weights('model-bw.h5')
print('Weights saved')

Model Saved
Weights saved
